In [1]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/fasttext-sbwc.3.6.e20.vec'

In [2]:
cantidad = 100000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [30]:
file= '/home/erwin/Genoma/cv-parser/resumes/prueba_genoma_experiencia.pdf'
file ='/home/erwin/Genoma/cv-parser/resumes/1574824594718-Ing._Comercial_Paula_Fetis_CV.pdf'
file ='/home/erwin/Genoma/cv-parser/resumes/1580248761709-CV_Sergio_Soto_Valdes.pdf'

In [41]:
def extract_text(path):
    '''
    Input: ruta hacia los archivos
    Salida: Texto plano como string
    '''
    with fitz.open(path) as doc:
        text = ""
        for page in doc:
            text += page.getText()
        # eliminar estos simbolos
        
        text_clean = text
        text_2 = ''
        for line in text_clean.splitlines():
            if not line.strip(): #si la linea esta vacia, saltar
                continue
            line_2=''
            for word in line.split():
                if word.isupper():
                    line_2 += word.capitalize()+' '

                else:
                    line_2 += word+ ' '
            text_2 += line_2 +'\n'
        
        simbolos = ' ,\n./:@'
        text_clean = ''
        for char in text_2:
            if (char.isalnum())| (char in simbolos):
                text_clean += char
        #print(text_clean)
        # Limpiar palabras completamente en mayusculas, es importante
        # para reconocer los nombres
        
        text = text_clean
    return text

In [42]:
text = extract_text(file)
print(text)

Sergio 
Soto 
Valdés 
56976821196sergiosoto94@gmail.com 
 
Vía 
Blanca 
7139, 
Vitacura. 
Resumen 
Profesional 
Ingeniero 
Comercial 
titulado 
de 
la 
Universidad 
del 
Desarrollo, 
mención 
en 
emprendimiento 
con 
manejo 
avanzado 
del 
inglés 
y 
experiencia 
en 
área 
comercial 
correspondiente 
a 
las 
prácticas 
profesionales 
realizadas 
en 
el 
extranjero. 
Me 
gusta 
trabajar 
equipo, 
siempre 
comprometido 
al 
cumplimiento 
de 
metas 
y 
objetivos, 
con 
alta 
capacidad 
de 
adaptación, 
buscando 
aprender 
lo 
máximo 
posible 
y 
dar 
lo 
mejor 
de 
si 
intereses 
enfocados 
en 
el 
área 
comercial 
y 
de 
marketing. 
Experiencia 
Laboral 
Septiembre 
2018 
a 
Diciembre 
2018 
Swapper. 
Valencia, 
España 
Sales 
Manager 
/ 
Práctica 
profesional. 
 
Responsable 
de 
la 
búsqueda 
y 
contacto 
con 
potenciales 
nuevos 
clientes. 
 
Apoyo 
al 
equipo 
en 
análisis 
comercial 
actualizando 
planillas 
de 
excel. 
 
Apoyo 
al 
equipo 
de 
marketing 
participando 
en 
brainstor

In [6]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [7]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.8897964


In [8]:
model.most_similar(positive=['experiencia'],topn=10)

[('experiencias', 0.7211438417434692),
 ('pericia', 0.6080396175384521),
 ('aprendidas', 0.5985227227210999),
 ('vivencia', 0.595109224319458),
 ('conocimientos', 0.5840999484062195),
 ('enriquecedora', 0.5687903761863708),
 ('adquirida', 0.5659759044647217),
 ('inexperiencia', 0.5602041482925415),
 ('aprendido', 0.5534920692443848),
 ('vivida', 0.5521409511566162)]

In [9]:
prc_description ='''Programador experto scripting desarrollo software experiencia cloud computing
responsable trabajo equipo carismatico google cloud aws procesamiento lenguaje natural nlp deep learning'''

In [10]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print(similar_words[i], word_value[similar_words[i]])
        print('------------------------------------------------')

programador 1
------------------------------------------------
desarrollador 0.7731334567070007
------------------------------------------------
programadores 0.6953645944595337
------------------------------------------------
experto 1
------------------------------------------------
especialista 0.7584600448608398
------------------------------------------------
experta 0.7457621097564697
------------------------------------------------
scripting 1
------------------------------------------------
scripts 0.7742691040039062
------------------------------------------------
script 0.7501298189163208
------------------------------------------------
desarrollo 1
------------------------------------------------
sostenible 0.6917796730995178
------------------------------------------------
socioeconómico 0.5877454280853271
------------------------------------------------
software 1
------------------------------------------------
hardware 0.8175531625747681
---------------------------------